In [1]:
import dill
import pandas as pd
from datetime import datetime

dict_studies = dill.load(open('data/dict_studies.pkd', 'rb'))

In [6]:
from collections import Counter
curr_dist=sorted(Counter([x['source'] for x in dict_studies]).items())
curr_dist

[('AVEO Pharmaceuticals, Inc.', 13),
 ('AbbVie', 56),
 ('Acceleron Pharma, Inc.', 3),
 ('Advaxis, Inc.', 8),
 ('Agenus Inc.', 13),
 ('Agios Pharmaceuticals, Inc.', 2),
 ('Amgen', 138),
 ('AstraZeneca', 229),
 ('Bristol-Myers Squibb', 251),
 ('Clovis Oncology, Inc.', 24),
 ('Eli Lilly and Company', 290),
 ('Exelixis', 15),
 ('Incyte Corporation', 70),
 ('Karyopharm Therapeutics Inc', 12),
 ('MorphoSys AG', 5),
 ('Pfizer', 196),
 ('Sanofi', 168),
 ('Seagen Inc.', 42),
 ('Spectrum Pharmaceuticals, Inc', 33),
 ('Ultragenyx Pharmaceutical Inc', 1),
 ('Verastem, Inc.', 14)]

In [21]:
ticker_names={'AstraZeneca':'AZN',
              'Sanofi':'SNY',
             'Amgen':'AMGN',
             'Incyte Corporation':'INCY',
             'Verastem, Inc.':'VSTM',
             'Bristol-Myers Squibb':'BMY',
             'Pfizer':'PFE',
             'Exelixis':'EXEL',
             'AbbVie':'ABBV',
             'Karyopharm Therapeutics Inc':'KPTI',
             'Seagen Inc.':'SGEN',
             'Spectrum Pharmaceuticals, Inc':'SPPI',
             'Eli Lilly and Company':'LLY',
             'Agenus Inc.':'AGEN',
             'Advaxis, Inc.':'ADXS',
             'Clovis Oncology, Inc.':'CLVS',
             'AVEO Pharmaceuticals, Inc.':'AVEO',
             'Acceleron Pharma, Inc.':'XLRN',
             'Agios Pharmaceuticals, Inc.':'AGIO',
             'MorphoSys AG':'MOR',
             'Ultragenyx Pharmaceutical Inc':'RARE',
             'Verastem, Inc.':'VSTM'}
name_tickers=dict(sorted([(x[1],x[0]) for x in ticker_names.items()]))
dill.dump(ticker_names, open('data/ticker_names.pkd', 'wb'))
dill.dump(name_tickers, open('data/name_tickers.pkd', 'wb'))

In [22]:
tickers=[x for x in name_tickers.keys()]
dill.dump(tickers, open('data/tickers.pkd', 'wb'))
tickers

['ABBV',
 'ADXS',
 'AGEN',
 'AGIO',
 'AMGN',
 'AVEO',
 'AZN',
 'BMY',
 'CLVS',
 'EXEL',
 'INCY',
 'KPTI',
 'LLY',
 'MOR',
 'PFE',
 'RARE',
 'SGEN',
 'SNY',
 'SPPI',
 'VSTM',
 'XLRN']

In [23]:
len(tickers)

21

In [24]:
df_dates=pd.DataFrame([([x[1] for x in y.items() if x[0].endswith('date') or x[0]=='nct_id' or x[0]=='brief_title' or x[0]=='source'] )for y in dict_studies],columns=['nct_id']+[x[0] for x in dict_studies[0].items() if x[0].endswith('date')]+['brief_title']+['source'])
dill.dump(df_dates, open('data/df_dates.pkd', 'wb'))

In [28]:
df_dates = dill.load(open('data/df_dates.pkd', 'rb'))
name_tickers = dill.load(open('data/name_tickers.pkd', 'rb'))
tickers = dill.load(open('data/tickers.pkd', 'rb'))

def extract_first__study_date(ticker):    
    ticker_dates=df_dates[df_dates.source==name_tickers[ticker]]
    ticker_dates.set_index('nct_id',inplace=True)
    df_sdates=pd.DataFrame(columns=['date','nct_id','datetype'])
    for col in ticker_dates.iloc[:,:-2].iteritems():
        df=pd.DataFrame()
        df['date']=col[1].values
        df['nct_id']=col[1].index
        df['datetype']=col[0]
        df_sdates=df_sdates.append(df)
    df_sdates=df_sdates.dropna().sort_values('date').set_index('date')

    first_date=datetime.strptime(str(df_sdates.index[0])[:10],'%Y-%m-%d')
    
    return ticker,first_date

first_study_dates=dict([extract_first__study_date(ticker) for ticker in tickers])
dill.dump(first_study_dates, open('data/first_study_dates.pkd', 'wb'))
first_study_dates

{'ABBV': datetime.datetime(2005, 3, 31, 0, 0),
 'ADXS': datetime.datetime(2014, 6, 13, 0, 0),
 'AGEN': datetime.datetime(2000, 3, 31, 0, 0),
 'AGIO': datetime.datetime(2016, 12, 5, 0, 0),
 'AMGN': datetime.datetime(1999, 7, 31, 0, 0),
 'AVEO': datetime.datetime(2007, 7, 16, 0, 0),
 'AZN': datetime.datetime(1994, 12, 31, 0, 0),
 'BMY': datetime.datetime(2000, 6, 30, 0, 0),
 'CLVS': datetime.datetime(2010, 5, 12, 0, 0),
 'EXEL': datetime.datetime(2007, 12, 7, 0, 0),
 'INCY': datetime.datetime(2007, 7, 31, 0, 0),
 'KPTI': datetime.datetime(2013, 12, 27, 0, 0),
 'LLY': datetime.datetime(1998, 9, 30, 0, 0),
 'MOR': datetime.datetime(2012, 9, 3, 0, 0),
 'PFE': datetime.datetime(1997, 2, 28, 0, 0),
 'RARE': datetime.datetime(2014, 11, 24, 0, 0),
 'SGEN': datetime.datetime(2000, 10, 31, 0, 0),
 'SNY': datetime.datetime(1996, 3, 31, 0, 0),
 'SPPI': datetime.datetime(1998, 10, 31, 0, 0),
 'VSTM': datetime.datetime(2013, 5, 31, 0, 0),
 'XLRN': datetime.datetime(2011, 10, 20, 0, 0)}

In [34]:
from utils import get_response, get_EOD_data
from datetime import timedelta
first_study_dates = dill.load(open('data/first_study_dates.pkd', 'rb'))
def get_stock_prices(daysdelta):
    ticker_first_dates={}
    for ticker,fdate in first_study_dates.items():
        try:
            first_date=datetime.strftime(fdate-timedelta(days=daysdelta),'%Y-%m-%d')
            last_date=str(datetime.now())[:10]
            ticker_first_dates[ticker]=fdate-timedelta(days=daysdelta)
            df_EOD=pd.DataFrame(get_EOD_data(ticker,first_date,last_date).json())
            df_EOD['date']=df_EOD['date'].apply(lambda x:x[:10]).apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
            df_EOD.set_index('date',inplace=True)
            dill.dump(df_EOD, open('data/EOD/'+ticker+'.pkd', 'wb'))
        except Exception as err:
            print(ticker,err)
    return ticker_first_dates
ticker_first_dates=get_stock_prices(30)
dill.dump(ticker_first_dates, open('data/ticker_first_dates.pkd', 'wb'))
ticker_first_dates

{'ABBV': datetime.datetime(2005, 3, 1, 0, 0),
 'ADXS': datetime.datetime(2014, 5, 14, 0, 0),
 'AGEN': datetime.datetime(2000, 3, 1, 0, 0),
 'AGIO': datetime.datetime(2016, 11, 5, 0, 0),
 'AMGN': datetime.datetime(1999, 7, 1, 0, 0),
 'AVEO': datetime.datetime(2007, 6, 16, 0, 0),
 'AZN': datetime.datetime(1994, 12, 1, 0, 0),
 'BMY': datetime.datetime(2000, 5, 31, 0, 0),
 'CLVS': datetime.datetime(2010, 4, 12, 0, 0),
 'EXEL': datetime.datetime(2007, 11, 7, 0, 0),
 'INCY': datetime.datetime(2007, 7, 1, 0, 0),
 'KPTI': datetime.datetime(2013, 11, 27, 0, 0),
 'LLY': datetime.datetime(1998, 8, 31, 0, 0),
 'MOR': datetime.datetime(2012, 8, 4, 0, 0),
 'PFE': datetime.datetime(1997, 1, 29, 0, 0),
 'RARE': datetime.datetime(2014, 10, 25, 0, 0),
 'SGEN': datetime.datetime(2000, 10, 1, 0, 0),
 'SNY': datetime.datetime(1996, 3, 1, 0, 0),
 'SPPI': datetime.datetime(1998, 10, 1, 0, 0),
 'VSTM': datetime.datetime(2013, 5, 1, 0, 0),
 'XLRN': datetime.datetime(2011, 9, 20, 0, 0)}

In [30]:
today=datetime.now()
str(today)[:10]

'2021-01-20'

In [3]:
df=dill.load(open('data/News/KPTI.pkd', 'rb'))
df

,date,text,yvalues
0,2021-01-12,News: SVB Leerink Maintains Outperform on Kary...,1
1,2021-01-11,News: Karyopharm Sees Prelim. Q4 Sales $35M-$3...,2
2,2021-01-11,Press Release: Karyopharm Announces Preliminar...,1
3,2021-01-11,News: Why Karyopharm Therapeutics Is Trading L...,2
4,2021-01-11,News: Karyopharm Therapeutics shares are tradi...,3
...,...,...,...
897,2013-12-02,News: Oppenheimer Initiates Coverage on Karyop...,1
898,2013-12-02,News: UPDATE: Bank of America Initiates Covera...,2
899,2013-12-02,News: Bank of America Initiates Coverage on Ka...,3
900,2013-11-06,News: IPO for Karyopharm Therapeutics Opens at...,4


In [11]:
df[df.text.str.find('Selinexor')!=-1]

,date,text,yvalues
5,2021-01-05,News: Karyopharm Therapeutics Partner Antengen...,4
8,2021-01-04,Press Release: Antengene Announces Submission ...,1
10,2021-01-03,Press Release: Antengene Submits NDA for ATG-0...,1
11,2020-12-20,Press Release: Antengene Announces its U.S. Pa...,2
14,2020-12-18,Press Release: Antengene Announces Approval of...,1
...,...,...,...
879,2014-04-24,News: Karyopharm Initiates Phase 2 Study of Se...,1
882,2014-04-21,News: Karyopharm Announces a Phase 1 Trial of ...,3
883,2014-04-15,News: Karyopharm Announces Initiation of a Pha...,1
892,2014-01-22,"News: Karyopharm Reports Data on Selinexor, Sa...",1


In [12]:
df[df.text.str.find('Verdinexor')!=-1]

,date,text,yvalues
576,2018-05-24,Press Release: Karyopharm and Antengene Sign E...,1
683,2017-05-03,News: Karyopharm and Anivive Lifesciences Sign...,1
684,2017-05-03,Press Release: Karyopharm and Anivive Lifescie...,2
